In [1]:
import re
import time
from tqdm import tqdm 
import pandas as pd
import requests

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd()  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

from config import urls_1140412


c:\PyWorkspace\presale-scraper\presale_scraper


In [ ]:
# # 預售屋建案查詢(2025/4/11)
# taipei = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/590f1fad4b56f04e9deed87e7f32c914?q=VTJGc2RHVmtYMTlyT2c0dkp3Mm1naitFeFljVndWVGFja2dGQVl3dkFRWXVNR3YwZXFMN05iZUthQ3dYMEQ3M3p0dldNZVdmSFM0S3dQdWNKRnJ0WG5LMjRNVXh5Smo2UGtqNXg0K2VsQTcrMHBReENhcHdVQzBLY040Q05TQXFsQWZWNkJhK2lkNG5nNUo4YTkvRy95ZU1uNjVMVmFRSDAreEVtVks0WG5nZUxXbHl2bStlY2tDdU5mdTBoWjd5MXE2MGpOUkxVQnJYRUtzYWpydXkxQ1JxMG8vT1NLR2pYSnFsdnZkdHhiYjRBa0p2WklES3BBZlVBQWlCenRsaFBmTVdGRXRZUXlKb2VxVGM4TGFhdnE2blZhQUx0ZzVYWFBoUzZwUWlBWEthSWVaQVZ1RWlXSlpkUE4zYnVCWXo="
# new_taipei = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/a8e282853cc329e3a49bf8fa49b6568f?q=VTJGc2RHVmtYMStOUlRzQVNnZk53MXpQOGtCQ0Z3R1pTN1E2SFdRc0J2ZzFVUlN2eHZwNk1xbXMxU25rNDFpU1dpUm4yVDA4SmFxN3NJVTNOd0ZQNkJMWmlGOXlWcS9MRmlKT2V0ZW9kL0NnaHdPaWhZMWU5Q1p6SWw0alNaTlRCSDZXMlRIamNnMnV5MVBuaVY4TXdTME42WldBUXhFOFhrREd3Y3J3VVFKNTR5VnFvVXpZcTByencwWW91SmNvc3NSNlFRV05wQ09KdzFRem1rTUh1QXFiR0UrZk0vekgyWGhoOVp3RjczNlM0NmRVQUZtTEV2VlplODZmcFB1eHNXRGN2Sy92bWRzTnMzbk5BZGVHd0lRUjNIS0dWQms0Qzl2aitxNUdkN2l6ME5ndFlMRTlSY2VGWE9PVk1oNGI="
# taoyuan = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/dfbf3a3dd66ee9285a4ed1145e75c9b6?q=VTJGc2RHVmtYMTk5b25UU2NDUXBNOXJWMCt2WWMxTE1nYmZxN0pURC8rZGdIN1RodUlML2dtRkdNbDhaVEIzcDluU3FLY2VhRmRCM1VCRmttb3hRbnJOWCs5QmVkK3VobGJTeU5VZStneGpaUFFYaHZkbkQ5c0NBTFhrUmZ3T1NkS0x2L09FSWQ1emR2aE5DRitDRE9KSW0wUE02OHNDdkZWTEJSUDFBVytIZDFYRzVMUVpGQzVYMnc0ckJyODB5VjlaOXNkR25yUGxtTlRhd0x5YzBVUC9YcWZQbXhjM1l1RHYyT1BCTDZVczR6ZGxSclcwUFZrdmhEeWhSWXNpTHpwT3RRalpZcWtYdU81b2FBVllDRDNTYkZyYTB5QWozeWlWSnJlZ2dyUG0vKzRlVnBwa284REJMR0N2OWcwQmc="
# hsinchu_city = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/a8aef2f6fd3421e0f0f4f36251ee59ba?q=VTJGc2RHVmtYMStuMytZL1pFKzg2TjRSb294TWpsdFhyR0Y0MTF2Vmo1bHdQL1ozMi9SMkVGOWtkZVFRY0ZsMlBFanp3MG9jOFpQTWdJUVRTVDREQXE4SEhRUGJMRVdLemdzYTNZZlV1MzV5MS9rdGVDWStOenRHMDJzL3hjb3FkM2pVTitBMWpYOUl1OWVvUHAzVGVnQXpIbVhVUFI2REQ4SGJEUXZVTGJuTVVhdVNDNWg5dDJNdXk4K2l6a0d4QjBySWdoMVdzMUdQZzVqa1NxcWtnTEU2OEJ5N2JENGxqbGJnZHEwNzA4VW93VEdmVnBXTzArOTI5dEw0TmVyR0laYkxrVy9TTVhBZDdmNjg2a09KVHhRUTBkbjZ5ZjBRelZxQXphQldyVHZueEJZM3ZiMGI2eHhuY245N0d0Nng="
# hsinchu_county = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/e87ca53ad3a268cd8f560e13de4021b0?q=VTJGc2RHVmtYMTg1WXI4ak44SGdXaTBOdHdYdHY0cEJRbVBkbkJGbFN1c21meHhlMHpuNjQyV3FrTGEzR0hNdXF2cC9Sa0thMTBudmpUdzBSNktnM1ZkWmg3VlhidFZ0UldXUElPdWl0UWpWOGZBaWRTdWt6U0FORnZLTHAyeS9FUkVnRjF0K0JjeXZXNThVL3FNNndMd1lyZ0YyaDRVdFE4d05SOHpCZjFMeE8wcGM0MWhLa2NMOGlpejdqaFM3M0szV2ViQXIzN2FBZkh1NzhkVmJzWktXeVFOWEVRQ0J6MmxhN25nK3k3UzhvQUpTRFNxWFBwV3hPOTJ4ZXFzYUVQM0o5dURuZHNtdWF3UUNacy9rZ1VMV0NoeWJ6RkNoTmJ1d2JjSCtlSlR4eEpBZnY4ekdRL0RqaUN3NVo3Zlc="
# taichung = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/0837c4242a3659d6442b9a560a6ff252?q=VTJGc2RHVmtYMTlaemduSTBxbXFwRVFIQ2Z4RkRpNGppR29uZGJxdU1EOHhQVWpGTEY4Y3NGclkrYy9CMmtRa0NzM0hMczdGTDhRWFJmemMyeFR0RGpIcFdYZzRhR0FxdW04MFZPUWtDVndtVVlZMmp4ZVNjS3k5TWFMQjlpc05wZFF5cXQzUUtycTQwOXZFTE90TG5FOFJCVUZmVGZxMlh2SkQ5dzRNQlhOL1FIc0wxUHFGamNOakFHNTdrSFlKaTl2VjA3Sk1LRU5KS3JIbU1VeFZnSU9hc2gzeVBsRmlFNWpjT1YzRzNSUXdrclpOdkhlTU03Ymgvd3lxS05HMzExWk9ueklvL2tmSWxSZm81R0FweDRnZk9GQXYzeXM5amJWK1FmQzVZekJvWUVpak9GSlRhZHBNZ0YvYnVXREc="
# tainan = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/0e642025931344a4ec851cb0948e81d3?q=VTJGc2RHVmtYMTlPeTVLTUdDRzRrMlk2SkQ3K3VxVUtvWlZMaUgwMEFQWU41U0Flekx2NWJ2LzhidktTV3YvaWJtVzFkb0I5dnpWQ2piRU0xQ2VZcnlLMHJtWmNDTlc0UmRhc2JNQU9ud2ZLWW1kZklVd0prbmdSQVZnVjM0RS9wM1dQYnhYa3E0MzE3dnU4VmhiMFliWTB6ZHR5d1RWTVBQcjIyaDlkaUhYb3ZCRkpDRmdDTjlWejh1YnFGRktTTmtUSEZYZytYcWI2TVJ6L0lQdHV1VElzbmJ1cHFtUTltZjVvN2VpRm54YzdKb3NLaytFelRhTUJRdmE4OHBvZXpXVGpiY1A3bmNkQWNOU1pBMU9CVmFCbFA1SXV3V0x0YXNmT0dvTTBubllpQkNqbjBqb09kL214eGZoVm9PYnU="
# kaohsiung = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/c3ebdb0abff21c7c1df8e759fae523d4?q=VTJGc2RHVmtYMS9Td3ZmZ3BnOW02UGtjWTdCekdPbVpUUC9SUmN1S3N4Nks0YkpaVDhjOFdNUWNQNm8wWjlDL3o1YWhEZ1V3ZzkycWc3ZldHQ091WEN0bWV5Q2Fhc0FJL1NwVDhsV3FCckRrT3grL201VkkxZnVDTDJnUWJYcXVBbFJHOWo4dHoyK0dGQzNQaHN0SHo4d1J5bHJGcUFMSFRVbGhYSFByaUN4b0pEMnVmTDlqaTB4dGV6L0M5c2ZTa0pBcFl4SVBmMkcyL2U3emI3Q1FsMzFsMzYyRGJ2VTFZT1Y3UzVtK2tFVk90SlR5cHNPYVAxMFlXL29sK21xZmYyM3pOd1dUeXpQYkU1S3JHY0hLcytlMmsyMlB4SU5RR05FbDRDaFRZNDFxZHJrL3lJU3A4RmdoZmFyS0RXdHg="
# keelung = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/3fcfceee4773076e77eb4f7acd71e48c?q=VTJGc2RHVmtYMSsrQVQ0ZWJyb0djbW0xRG5DZ0dEQUMwUXJ5YXpBVXdyL2lzTld4M0sxR1hRdHkvTjIvd3Y5UTN6OXcrbTZPYnd5SUhLQVk3eDRmRVRaajFNSmd6R3MrdnJ4SjJ6K0s2SWphYlZ3OFpLQUtvKzBNZVc0empVZC9qRUgxUWdsTFVHQmFVRzk4QzRXd1JiM0J5YVA5aUF2alBaV0VPQmxLTk5haEdyZDJxalNaaDRJMGJOTUc0SWFoUmN4aXR0b3ZZUHkvMkZtVnlySERNOEROcXBiOW1KN0N2bGozTi9JUiszWTd4VHFwNFJ1eWpxejR2U1pTZGNJb29NWlVqSnc4elBoaTR4b2JhUnEyY1V6b2FMOFRvU2FhZDFBbXJ1M1BFZVdUZGdDRXlNVWdrOTIwOGxDV1JsK28="
# chiayi_city = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/6c0818790c0f1b5a50fcb1d2afbd5472?q=VTJGc2RHVmtYMStQeHorMU5FZmFSWGszdnp2V015aEJsUm9TeUdvSGoyVUMvQ1ZCdlNsUlRyM2JiM29ab0p5UkJYcFhWTjk5WEZQZE45dmcwVE1MMlFEWVdBdUthOExlUHpNdE5BL1NBbzdidGZHOWRDaVV0VlNhY0tURGVWb3JYSDZLYTNtMjJ0WDlZKytVcDA2RDNMajErN2xvYVo4TDhZcW5naDMzTWhaSDdwVkhXMFFETWNjRldKcnFOYzZnSVpZMTc1cndrU2tscERBUmpJZnd1ci9pSVRhZzBxY3R1bHZ0TWdBRW1pUTRjYlgxanpoaXBxOXUxNWh6QXQzQjhTVXFoc2pVL1czTzFPT3RKTkVMRFpsc0dpK1VLenRoQnpsMjRxbWlSWTRtSGY4WStyYUpoa3ZQRXBCR3g3OGI="
# yilan = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/fc0fe973a7b4e0b3f2f1f71d97ee8488?q=VTJGc2RHVmtYMS9UUWlaYTRwOXVNUEp3MjhsTmI3T1BBZmxKSjRPSGhTalpUUEF5YnFXVUtPY3RWY2VvMldrWHd0c29zc2JIeFhWaGZkZE1FOUI2THVTZWFlM2RDVVpremtuQW5uVEJSS1VmM0NJakpHblo3RGNjeU1TVXA3WjhXUVpCYnRmT1JlTGJ6OUhqbEZ5aE1SN3VaNmNLOGVnbDRBTG42L3lIWlJ2MEtZTnBVYVJkS2d0cWZtU2F1RFl6Nm5qWjlvcTNpS1VBQVlORllMYjZzcU1TNWJ4aHZLVnR5SHdDc1QwdXNtK2F3ajZVRUtKSCtxeHhXUzZiakFqY0h4blFqSDlIR01FY3pWREFHcnhudVJtbi9UZWxncHAzQ0NBNGpoMUJEME02dWNUVmRHYUVZNUkrZ3lKaE5FZXE="  
# miaoli = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/ea88499f21a09c39cfb32ba66852f9bd?q=VTJGc2RHVmtYMTk4OEh3N1hIc1N4K2RrdS9RZzFZZUFqQ2FpL05rTnJQQ3p3STU2VnQ3b0ZscEhWbkx5dmRRLzkwK0lyekhoM1l1TFo3RGN0NWRZWUo4S2NWWDVveCttSGVWS09mZFpxdnViSDFwc0VyS1B6a1c2WlNmNk1sYmJtd1Fhd2lUbWlDYzJIWUp0QjIwT2RqeFFUOEJUWHBmQVpJa0hRbzh0dWErSURabTBqVEE5akwxRzJvN0F3dzY1R3RVMEMwMjlYYzBLUnZ0MVl2WkJNbTE5eFd0dmgveUhrWHBxZlJuZVByNmNJajlHeUtVSDVGeVA1N2w5QnB4dlhBck9NUWJQTmZMSEhwMmtzcWlJNk1hWGRSbGFPcEJBZHBiSjFudmZ1TlFRS1RRTyt1c1MxdnEvUVg5aG9JcUg="
# changhua = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/4d038ed90563ca35f2333ce2493d401d?q=VTJGc2RHVmtYMTlWdCtoZTRaVEo4dWJpYXZmRFZWcWUwWFNQdTFkdDBvWWVucVlvK3VkMWpaOG4rK1Z4ZWRoSzNhTkY1di9CdEF6VW1NSmU1RUJnbFJEQS9PL0VrY2ZMc1hDdUwrZjYzQU5QaU4rSHVCRTdUNkY5dkRFQTRDS0hGVFNXMCtUOUdkbkV3STk3REUrRFg5RnBqR3BPNlN1c1JKWUxrbCtzT1F2Vmg0YzkyYjRMZTJQbkFxQjJDNFNlTkRUcjc2NnNRVGNQUVRXSk5ueGZrOG9Td3RpOFgwOHJpb2FURTNQNVJIV1ZyZHhiV1JlYkZ1eU9MV0NZVy9TbisvNXRWNTBBdUVEdUgyR1ZFaHdReVVlY056cDJ4ZGdmOXdwdDlzZitjbWdYS0U4Z3daUEpob3dWTzFYM0RiazU=" 
# nantou = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/1ec0e19e86896b0e11668aa4832e73f9?q=VTJGc2RHVmtYMS9CWjRnMzZYKzQydk5VSjFyaFdTc2loQjYzREpKTEZTTHBsRXZaOG9RT2RIY0lRd2V3YVNZL1lJTk1Ib3E2cVhRNDFGQzhqRTVONXE1SUFoYkxCbmpHZGZlSGRFZ0hkamQwNFdLeExjSEFEU1dHYlE1TktGQVZoZWlNS3BBWC9Jb29JQ1oxNllYTjlQSHY3bXdqVUdJVVJzSmV1Y1RPajFwWkZmV2M2TVNRUHFWV1lBMjdrWkw3ZXJzbFQ1SnN6L1dWZ1BpaTY0NGU5UlcwOThKQ1h2aHNSMFB3QjNNaG1waXN4UElNZkRZU3BubFN6T1gzUUxmR1FHbVFhVUdWYlJvUUNrUHhmZXA1R1lldnU1YWplaGFKTWxIeWkvQUxDNUF4RDdPU2NOZHVoQWIya0wwRzE0R2o="
# yunlin = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/5cb21bf504adf65943a3e28b653d55f9?q=VTJGc2RHVmtYMS85UFBkRk16RWxFdGltLytiaG1vZ2o2aGVnZHZHR0dZWEU4eWhaaXpqMlB3UGVuTDArYTVPd25PQ2cvZUgwRGVzWDN3dFpDQjM0VnR4U29aSW5GZWluVks2cjZaRUFkOUM2QWJWK3ZUOFU3VGpJUHZiNWRVRXc3Y2h4dnFsckJBVndpMEE3VFZ3Y1A2bGc5TUc5anVNRHNEeGc3bmF0L1kyREJhZXpRMzZUeDJnUExoZ3NuaXZsSlV4T1RBbHI3Uk1uZlJmKzFxdjFMenI3ekRnWWRSREVnWjh4YU9ybWV1NXNCM3QrUStZT3hzSjJrUjh0R1U5SkdzQlhVR1hZclY2dXhwVDd6TEU2SWsvT1V4ZTlMK1EzL28vS1BVTldhOVh3MTc2RUZyRWpiL25NNkZUUFBIVnQ="
# chiayi_county = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/93b64316954079655a1f3f6bbf4a58a8?q=VTJGc2RHVmtYMSt3eW41SmE3ME1LUVhyWTB1NXhNYlhvYlFFOHA3VHkwQkZqWThidWpkSjJjU2k4UkkvWkpMbFNVb3NoZTZ2N1duVnFIOWxuQ0FBUnlnUkVYOGN1Rk5xdWNlZEd6Q0xFZVdORjVPR2NBMUZMbzVxczdBQVA5S2pJR0lLZGlkdDJ0SzltOG5yVWNLNjlaRkxvVEVId2h0SHRCN21LY0craWFZQ2M3ZEk2OGhTSUFZNEV3RFpQcHR0NzYxalMvSjNkcFhJOWMrbTNOZzAyOG4wbkJlZHZ6UWVJcXArcUt0b3poL25DK1VtbmhXQU9JdmNVeHR4VHBqaHBsa3pia2xXY3U3anlJWTlYS3R5SndtSHp2TzgxV09nMi8wN2FoRVdOU2tubjF1TEZnaXBQSXFldHZBbURYYSs="
# pingtung = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/c6e86f5f1d44360781c3e51fc7a5fb22?q=VTJGc2RHVmtYMS9NVXM0bUhyQlFFTzRLbjVzQStCQzcyR3l0d1N1cGk1U29jSktDd1BZSTBYUUFZckhHa1VYbEhXT29iUHNmVkJUeGpFdnRlOUhIazU0OWkyRXhBTXV6SHd0L1Fjc2gwV1laQ0MySGg0OEJnWU5Zc280YW1LL1lPWHA4OWFUekNLOXp6YzlUUzNOL3VycittL3lJUVA4VEw0cXdxNFJxdFE1b1NPTytOUmJhY0VPdnBUcStHSkNzemFHN0V0QnM1bHFiT1l5enp6a2lKeVhFbGw1ZWdPWjhoZDlwa1ZVam00VUxxOHR6V1VlUWRLajY0cE1Xc1ZZOGVIYTNWOG1wTllWV0lsYVlTaXRyeU9lUER0a0R1WG9UaHlvUzd3R0puVUpWK2hFaU9NQ05lVU1FSlVUeCsxOXM="
# taitung = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/5f00ba2284ad4f8b31db0e6fe95c96a7?q=VTJGc2RHVmtYMS9teFNDUmR6dDRQNTdNOXJDVkV0TURkemVrVUVBYWRuTWFYODF3T2dLc1o4K091ZDZNUzBLZ3QrUjdUWG9IVGNRMm96ZDI4VzFZU1hrQVJOc3RMRGE3NnREeFl1VW5lOXZFQUVnNjBkM2RwbWtZYlBHaURSMzV3QVhETDFhRkFMUlV3R1Q2ZExlalp4eEcrWVNPWFlhQlFxcUNhZG5JZFdnNUdVL2szR3BqeCtzQmZ2anpqc1o3VUM2T0tYcUU4K1FRSElhZmx0dXc2cGp1bHk2d2haQU90U1hkbWFZQStkZUFrSE5ZRXFIbXJnMVlZWm1XTzJETElLWFJYVDBpdFN3YWNsVGhSMG9EaU8rd0RubEorcXVYVytWZ3pvYWpSSmY4bUdOTlA2VzMvWUtiL2hua0ZVM1c="
# hualien = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/107f2b57838510c356e3b5d9b660e606?q=VTJGc2RHVmtYMTkyajZiTzAraEtITjl1SFdtK2ZSeTdwU3cwckthalM5YkVCLzRRcHJLd09kWWNzd2RIYmxzQWFzTW9GVUFoZzdzTXZtS3dXNktMZjlITG9BeCt2MWxmdHFLbkNweUpuS3IwWjZKYU0xSTRHZ21nWWQrUFpmckhWK3ZNc1NUV3lhRkFsb1dqcUx5QVFoaEF0SURNVXpPY0hJOVMrMTBmcDNoYU40bDBVNEVuMXd3U1duN1VqSmZGT0hsWm9yZzErdEc1L3NWd0FiU2lmRngvamVSb3ZCazViYzlOcHZvYmxydnlCWTdtd3hwYnZLRUVuN1dsMDd3RnI0YWVuRkxhckZHVXN2bUErUlYwSHpJblk5YVZuaC9JWjJULzRDbEVtMlUwc3BXRU0zUE1OcFVTNWptdWFRVmU="



In [3]:
res = requests.get("https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/590f1fad4b56f04e9deed87e7f32c914?q=VTJGc2RHVmtYMTlyT2c0dkp3Mm1naitFeFljVndWVGFja2dGQVl3dkFRWXVNR3YwZXFMN05iZUthQ3dYMEQ3M3p0dldNZVdmSFM0S3dQdWNKRnJ0WG5LMjRNVXh5Smo2UGtqNXg0K2VsQTcrMHBReENhcHdVQzBLY040Q05TQXFsQWZWNkJhK2lkNG5nNUo4YTkvRy95ZU1uNjVMVmFRSDAreEVtVks0WG5nZUxXbHl2bStlY2tDdU5mdTBoWjd5MXE2MGpOUkxVQnJYRUtzYWpydXkxQ1JxMG8vT1NLR2pYSnFsdnZkdHhiYjRBa0p2WklES3BBZlVBQWlCenRsaFBmTVdGRXRZUXlKb2VxVGM4TGFhdnE2blZhQUx0ZzVYWFBoUzZwUWlBWEthSWVaQVZ1RWlXSlpkUE4zYnVCWXo=")
res

<Response [200]>

In [2]:
urls = {
    "臺北市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/590f1fad4b56f04e9deed87e7f32c914?q=VTJGc2RHVmtYMTlyT2c0dkp3Mm1naitFeFljVndWVGFja2dGQVl3dkFRWXVNR3YwZXFMN05iZUthQ3dYMEQ3M3p0dldNZVdmSFM0S3dQdWNKRnJ0WG5LMjRNVXh5Smo2UGtqNXg0K2VsQTcrMHBReENhcHdVQzBLY040Q05TQXFsQWZWNkJhK2lkNG5nNUo4YTkvRy95ZU1uNjVMVmFRSDAreEVtVks0WG5nZUxXbHl2bStlY2tDdU5mdTBoWjd5MXE2MGpOUkxVQnJYRUtzYWpydXkxQ1JxMG8vT1NLR2pYSnFsdnZkdHhiYjRBa0p2WklES3BBZlVBQWlCenRsaFBmTVdGRXRZUXlKb2VxVGM4TGFhdnE2blZhQUx0ZzVYWFBoUzZwUWlBWEthSWVaQVZ1RWlXSlpkUE4zYnVCWXo=",
    # "新北市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/a8e282853cc329e3a49bf8fa49b6568f?q=VTJGc2RHVmtYMStOUlRzQVNnZk53MXpQOGtCQ0Z3R1pTN1E2SFdRc0J2ZzFVUlN2eHZwNk1xbXMxU25rNDFpU1dpUm4yVDA4SmFxN3NJVTNOd0ZQNkJMWmlGOXlWcS9MRmlKT2V0ZW9kL0NnaHdPaWhZMWU5Q1p6SWw0alNaTlRCSDZXMlRIamNnMnV5MVBuaVY4TXdTME42WldBUXhFOFhrREd3Y3J3VVFKNTR5VnFvVXpZcTByencwWW91SmNvc3NSNlFRV05wQ09KdzFRem1rTUh1QXFiR0UrZk0vekgyWGhoOVp3RjczNlM0NmRVQUZtTEV2VlplODZmcFB1eHNXRGN2Sy92bWRzTnMzbk5BZGVHd0lRUjNIS0dWQms0Qzl2aitxNUdkN2l6ME5ndFlMRTlSY2VGWE9PVk1oNGI=",
    # "桃園市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/dfbf3a3dd66ee9285a4ed1145e75c9b6?q=VTJGc2RHVmtYMTk5b25UU2NDUXBNOXJWMCt2WWMxTE1nYmZxN0pURC8rZGdIN1RodUlML2dtRkdNbDhaVEIzcDluU3FLY2VhRmRCM1VCRmttb3hRbnJOWCs5QmVkK3VobGJTeU5VZStneGpaUFFYaHZkbkQ5c0NBTFhrUmZ3T1NkS0x2L09FSWQ1emR2aE5DRitDRE9KSW0wUE02OHNDdkZWTEJSUDFBVytIZDFYRzVMUVpGQzVYMnc0ckJyODB5VjlaOXNkR25yUGxtTlRhd0x5YzBVUC9YcWZQbXhjM1l1RHYyT1BCTDZVczR6ZGxSclcwUFZrdmhEeWhSWXNpTHpwT3RRalpZcWtYdU81b2FBVllDRDNTYkZyYTB5QWozeWlWSnJlZ2dyUG0vKzRlVnBwa284REJMR0N2OWcwQmc=",
    # "新竹市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/a8aef2f6fd3421e0f0f4f36251ee59ba?q=VTJGc2RHVmtYMStuMytZL1pFKzg2TjRSb294TWpsdFhyR0Y0MTF2Vmo1bHdQL1ozMi9SMkVGOWtkZVFRY0ZsMlBFanp3MG9jOFpQTWdJUVRTVDREQXE4SEhRUGJMRVdLemdzYTNZZlV1MzV5MS9rdGVDWStOenRHMDJzL3hjb3FkM2pVTitBMWpYOUl1OWVvUHAzVGVnQXpIbVhVUFI2REQ4SGJEUXZVTGJuTVVhdVNDNWg5dDJNdXk4K2l6a0d4QjBySWdoMVdzMUdQZzVqa1NxcWtnTEU2OEJ5N2JENGxqbGJnZHEwNzA4VW93VEdmVnBXTzArOTI5dEw0TmVyR0laYkxrVy9TTVhBZDdmNjg2a09KVHhRUTBkbjZ5ZjBRelZxQXphQldyVHZueEJZM3ZiMGI2eHhuY245N0d0Nng=",
    # "新竹縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/e87ca53ad3a268cd8f560e13de4021b0?q=VTJGc2RHVmtYMTg1WXI4ak44SGdXaTBOdHdYdHY0cEJRbVBkbkJGbFN1c21meHhlMHpuNjQyV3FrTGEzR0hNdXF2cC9Sa0thMTBudmpUdzBSNktnM1ZkWmg3VlhidFZ0UldXUElPdWl0UWpWOGZBaWRTdWt6U0FORnZLTHAyeS9FUkVnRjF0K0JjeXZXNThVL3FNNndMd1lyZ0YyaDRVdFE4d05SOHpCZjFMeE8wcGM0MWhLa2NMOGlpejdqaFM3M0szV2ViQXIzN2FBZkh1NzhkVmJzWktXeVFOWEVRQ0J6MmxhN25nK3k3UzhvQUpTRFNxWFBwV3hPOTJ4ZXFzYUVQM0o5dURuZHNtdWF3UUNacy9rZ1VMV0NoeWJ6RkNoTmJ1d2JjSCtlSlR4eEpBZnY4ekdRL0RqaUN3NVo3Zlc=",
    # "臺中市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/0837c4242a3659d6442b9a560a6ff252?q=VTJGc2RHVmtYMTlaemduSTBxbXFwRVFIQ2Z4RkRpNGppR29uZGJxdU1EOHhQVWpGTEY4Y3NGclkrYy9CMmtRa0NzM0hMczdGTDhRWFJmemMyeFR0RGpIcFdYZzRhR0FxdW04MFZPUWtDVndtVVlZMmp4ZVNjS3k5TWFMQjlpc05wZFF5cXQzUUtycTQwOXZFTE90TG5FOFJCVUZmVGZxMlh2SkQ5dzRNQlhOL1FIc0wxUHFGamNOakFHNTdrSFlKaTl2VjA3Sk1LRU5KS3JIbU1VeFZnSU9hc2gzeVBsRmlFNWpjT1YzRzNSUXdrclpOdkhlTU03Ymgvd3lxS05HMzExWk9ueklvL2tmSWxSZm81R0FweDRnZk9GQXYzeXM5amJWK1FmQzVZekJvWUVpak9GSlRhZHBNZ0YvYnVXREc=",
    # "臺南市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/0e642025931344a4ec851cb0948e81d3?q=VTJGc2RHVmtYMTlPeTVLTUdDRzRrMlk2SkQ3K3VxVUtvWlZMaUgwMEFQWU41U0Flekx2NWJ2LzhidktTV3YvaWJtVzFkb0I5dnpWQ2piRU0xQ2VZcnlLMHJtWmNDTlc0UmRhc2JNQU9ud2ZLWW1kZklVd0prbmdSQVZnVjM0RS9wM1dQYnhYa3E0MzE3dnU4VmhiMFliWTB6ZHR5d1RWTVBQcjIyaDlkaUhYb3ZCRkpDRmdDTjlWejh1YnFGRktTTmtUSEZYZytYcWI2TVJ6L0lQdHV1VElzbmJ1cHFtUTltZjVvN2VpRm54YzdKb3NLaytFelRhTUJRdmE4OHBvZXpXVGpiY1A3bmNkQWNOU1pBMU9CVmFCbFA1SXV3V0x0YXNmT0dvTTBubllpQkNqbjBqb09kL214eGZoVm9PYnU=",
    # "高雄市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/c3ebdb0abff21c7c1df8e759fae523d4?q=VTJGc2RHVmtYMS9Td3ZmZ3BnOW02UGtjWTdCekdPbVpUUC9SUmN1S3N4Nks0YkpaVDhjOFdNUWNQNm8wWjlDL3o1YWhEZ1V3ZzkycWc3ZldHQ091WEN0bWV5Q2Fhc0FJL1NwVDhsV3FCckRrT3grL201VkkxZnVDTDJnUWJYcXVBbFJHOWo4dHoyK0dGQzNQaHN0SHo4d1J5bHJGcUFMSFRVbGhYSFByaUN4b0pEMnVmTDlqaTB4dGV6L0M5c2ZTa0pBcFl4SVBmMkcyL2U3emI3Q1FsMzFsMzYyRGJ2VTFZT1Y3UzVtK2tFVk90SlR5cHNPYVAxMFlXL29sK21xZmYyM3pOd1dUeXpQYkU1S3JHY0hLcytlMmsyMlB4SU5RR05FbDRDaFRZNDFxZHJrL3lJU3A4RmdoZmFyS0RXdHg=",
    # "基隆市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/3fcfceee4773076e77eb4f7acd71e48c?q=VTJGc2RHVmtYMSsrQVQ0ZWJyb0djbW0xRG5DZ0dEQUMwUXJ5YXpBVXdyL2lzTld4M0sxR1hRdHkvTjIvd3Y5UTN6OXcrbTZPYnd5SUhLQVk3eDRmRVRaajFNSmd6R3MrdnJ4SjJ6K0s2SWphYlZ3OFpLQUtvKzBNZVc0empVZC9qRUgxUWdsTFVHQmFVRzk4QzRXd1JiM0J5YVA5aUF2alBaV0VPQmxLTk5haEdyZDJxalNaaDRJMGJOTUc0SWFoUmN4aXR0b3ZZUHkvMkZtVnlySERNOEROcXBiOW1KN0N2bGozTi9JUiszWTd4VHFwNFJ1eWpxejR2U1pTZGNJb29NWlVqSnc4elBoaTR4b2JhUnEyY1V6b2FMOFRvU2FhZDFBbXJ1M1BFZVdUZGdDRXlNVWdrOTIwOGxDV1JsK28=",
    # "嘉義市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/6c0818790c0f1b5a50fcb1d2afbd5472?q=VTJGc2RHVmtYMStQeHorMU5FZmFSWGszdnp2V015aEJsUm9TeUdvSGoyVUMvQ1ZCdlNsUlRyM2JiM29ab0p5UkJYcFhWTjk5WEZQZE45dmcwVE1MMlFEWVdBdUthOExlUHpNdE5BL1NBbzdidGZHOWRDaVV0VlNhY0tURGVWb3JYSDZLYTNtMjJ0WDlZKytVcDA2RDNMajErN2xvYVo4TDhZcW5naDMzTWhaSDdwVkhXMFFETWNjRldKcnFOYzZnSVpZMTc1cndrU2tscERBUmpJZnd1ci9pSVRhZzBxY3R1bHZ0TWdBRW1pUTRjYlgxanpoaXBxOXUxNWh6QXQzQjhTVXFoc2pVL1czTzFPT3RKTkVMRFpsc0dpK1VLenRoQnpsMjRxbWlSWTRtSGY4WStyYUpoa3ZQRXBCR3g3OGI=",
    # "宜蘭縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/fc0fe973a7b4e0b3f2f1f71d97ee8488?q=VTJGc2RHVmtYMS9UUWlaYTRwOXVNUEp3MjhsTmI3T1BBZmxKSjRPSGhTalpUUEF5YnFXVUtPY3RWY2VvMldrWHd0c29zc2JIeFhWaGZkZE1FOUI2THVTZWFlM2RDVVpremtuQW5uVEJSS1VmM0NJakpHblo3RGNjeU1TVXA3WjhXUVpCYnRmT1JlTGJ6OUhqbEZ5aE1SN3VaNmNLOGVnbDRBTG42L3lIWlJ2MEtZTnBVYVJkS2d0cWZtU2F1RFl6Nm5qWjlvcTNpS1VBQVlORllMYjZzcU1TNWJ4aHZLVnR5SHdDc1QwdXNtK2F3ajZVRUtKSCtxeHhXUzZiakFqY0h4blFqSDlIR01FY3pWREFHcnhudVJtbi9UZWxncHAzQ0NBNGpoMUJEME02dWNUVmRHYUVZNUkrZ3lKaE5FZXE=",
    # "苗栗縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/ea88499f21a09c39cfb32ba66852f9bd?q=VTJGc2RHVmtYMTk4OEh3N1hIc1N4K2RrdS9RZzFZZUFqQ2FpL05rTnJQQ3p3STU2VnQ3b0ZscEhWbkx5dmRRLzkwK0lyekhoM1l1TFo3RGN0NWRZWUo4S2NWWDVveCttSGVWS09mZFpxdnViSDFwc0VyS1B6a1c2WlNmNk1sYmJtd1Fhd2lUbWlDYzJIWUp0QjIwT2RqeFFUOEJUWHBmQVpJa0hRbzh0dWErSURabTBqVEE5akwxRzJvN0F3dzY1R3RVMEMwMjlYYzBLUnZ0MVl2WkJNbTE5eFd0dmgveUhrWHBxZlJuZVByNmNJajlHeUtVSDVGeVA1N2w5QnB4dlhBck9NUWJQTmZMSEhwMmtzcWlJNk1hWGRSbGFPcEJBZHBiSjFudmZ1TlFRS1RRTyt1c1MxdnEvUVg5aG9JcUg=",
    # "彰化縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/4d038ed90563ca35f2333ce2493d401d?q=VTJGc2RHVmtYMTlWdCtoZTRaVEo4dWJpYXZmRFZWcWUwWFNQdTFkdDBvWWVucVlvK3VkMWpaOG4rK1Z4ZWRoSzNhTkY1di9CdEF6VW1NSmU1RUJnbFJEQS9PL0VrY2ZMc1hDdUwrZjYzQU5QaU4rSHVCRTdUNkY5dkRFQTRDS0hGVFNXMCtUOUdkbkV3STk3REUrRFg5RnBqR3BPNlN1c1JKWUxrbCtzT1F2Vmg0YzkyYjRMZTJQbkFxQjJDNFNlTkRUcjc2NnNRVGNQUVRXSk5ueGZrOG9Td3RpOFgwOHJpb2FURTNQNVJIV1ZyZHhiV1JlYkZ1eU9MV0NZVy9TbisvNXRWNTBBdUVEdUgyR1ZFaHdReVVlY056cDJ4ZGdmOXdwdDlzZitjbWdYS0U4Z3daUEpob3dWTzFYM0RiazU=",
    # "南投縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/341cd3107a94ed892fe54a88b0cf19bb?q=VTJGc2RHVmtYMStybGhvZjBld3J3WUVxTUF3bTlTM21uci9aUDdLYjhwa1hMditnL0UvYUc0dllDSTRIU2w5c21FZjIranVVblpONWdHSVY4Wi9HdGhjdm51NEhKT1k4eDZBT2N0RUkzVnk4azhlYmJmVjI3S0t3bUd5ZVRWNlZ0bVQ5bTd3aUNJWUVmdkp1NjlSSkdiSnNnQTAvVUVXNnFWdVFEcHdYWmVqWHRyUTMrNU5IcEl6RWVMV0dVUWI0aXFUc0ZXRG94YnFjWnIwZjhzRmtBUTZqUXFzS1VOSndKRTgxYWtrMFk5cEU1dUR0cTJDZ1hjVTlFUWxLQWczMURtRXY0WFR2cEMrK2RXNTZENGptWXZFUW1QVmxSVHVCdDh0SjkxRnFoUlo5KzJVWnRCbUc0TUxlMWpIWHhodVo=",
    # "雲林縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/5cb21bf504adf65943a3e28b653d55f9?q=VTJGc2RHVmtYMS85UFBkRk16RWxFdGltLytiaG1vZ2o2aGVnZHZHR0dZWEU4eWhaaXpqMlB3UGVuTDArYTVPd25PQ2cvZUgwRGVzWDN3dFpDQjM0VnR4U29aSW5GZWluVks2cjZaRUFkOUM2QWJWK3ZUOFU3VGpJUHZiNWRVRXc3Y2h4dnFsckJBVndpMEE3VFZ3Y1A2bGc5TUc5anVNRHNEeGc3bmF0L1kyREJhZXpRMzZUeDJnUExoZ3NuaXZsSlV4T1RBbHI3Uk1uZlJmKzFxdjFMenI3ekRnWWRSREVnWjh4YU9ybWV1NXNCM3QrUStZT3hzSjJrUjh0R1U5SkdzQlhVR1hZclY2dXhwVDd6TEU2SWsvT1V4ZTlMK1EzL28vS1BVTldhOVh3MTc2RUZyRWpiL25NNkZUUFBIVnQ=",
    # "嘉義縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/93b64316954079655a1f3f6bbf4a58a8?q=VTJGc2RHVmtYMSt3eW41SmE3ME1LUVhyWTB1NXhNYlhvYlFFOHA3VHkwQkZqWThidWpkSjJjU2k4UkkvWkpMbFNVb3NoZTZ2N1duVnFIOWxuQ0FBUnlnUkVYOGN1Rk5xdWNlZEd6Q0xFZVdORjVPR2NBMUZMbzVxczdBQVA5S2pJR0lLZGlkdDJ0SzltOG5yVWNLNjlaRkxvVEVId2h0SHRCN21LY0craWFZQ2M3ZEk2OGhTSUFZNEV3RFpQcHR0NzYxalMvSjNkcFhJOWMrbTNOZzAyOG4wbkJlZHZ6UWVJcXArcUt0b3poL25DK1VtbmhXQU9JdmNVeHR4VHBqaHBsa3pia2xXY3U3anlJWTlYS3R5SndtSHp2TzgxV09nMi8wN2FoRVdOU2tubjF1TEZnaXBQSXFldHZBbURYYSs=",
    # "屏東縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/c6e86f5f1d44360781c3e51fc7a5fb22?q=VTJGc2RHVmtYMS9NVXM0bUhyQlFFTzRLbjVzQStCQzcyR3l0d1N1cGk1U29jSktDd1BZSTBYUUFZckhHa1VYbEhXT29iUHNmVkJUeGpFdnRlOUhIazU0OWkyRXhBTXV6SHd0L1Fjc2gwV1laQ0MySGg0OEJnWU5Zc280YW1LL1lPWHA4OWFUekNLOXp6YzlUUzNOL3VycittL3lJUVA4VEw0cXdxNFJxdFE1b1NPTytOUmJhY0VPdnBUcStHSkNzemFHN0V0QnM1bHFiT1l5enp6a2lKeVhFbGw1ZWdPWjhoZDlwa1ZVam00VUxxOHR6V1VlUWRLajY0cE1Xc1ZZOGVIYTNWOG1wTllWV0lsYVlTaXRyeU9lUER0a0R1WG9UaHlvUzd3R0puVUpWK2hFaU9NQ05lVU1FSlVUeCsxOXM=",
    # "臺東縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/5f00ba2284ad4f8b31db0e6fe95c96a7?q=VTJGc2RHVmtYMS9teFNDUmR6dDRQNTdNOXJDVkV0TURkemVrVUVBYWRuTWFYODF3T2dLc1o4K091ZDZNUzBLZ3QrUjdUWG9IVGNRMm96ZDI4VzFZU1hrQVJOc3RMRGE3NnREeFl1VW5lOXZFQUVnNjBkM2RwbWtZYlBHaURSMzV3QVhETDFhRkFMUlV3R1Q2ZExlalp4eEcrWVNPWFlhQlFxcUNhZG5JZFdnNUdVL2szR3BqeCtzQmZ2anpqc1o3VUM2T0tYcUU4K1FRSElhZmx0dXc2cGp1bHk2d2haQU90U1hkbWFZQStkZUFrSE5ZRXFIbXJnMVlZWm1XTzJETElLWFJYVDBpdFN3YWNsVGhSMG9EaU8rd0RubEorcXVYVytWZ3pvYWpSSmY4bUdOTlA2VzMvWUtiL2hua0ZVM1c=",
    # "花蓮縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/107f2b57838510c356e3b5d9b660e606?q=VTJGc2RHVmtYMTkyajZiTzAraEtITjl1SFdtK2ZSeTdwU3cwckthalM5YkVCLzRRcHJLd09kWWNzd2RIYmxzQWFzTW9GVUFoZzdzTXZtS3dXNktMZjlITG9BeCt2MWxmdHFLbkNweUpuS3IwWjZKYU0xSTRHZ21nWWQrUFpmckhWK3ZNc1NUV3lhRkFsb1dqcUx5QVFoaEF0SURNVXpPY0hJOVMrMTBmcDNoYU40bDBVNEVuMXd3U1duN1VqSmZGT0hsWm9yZzErdEc1L3NWd0FiU2lmRngvamVSb3ZCazViYzlOcHZvYmxydnlCWTdtd3hwYnZLRUVuN1dsMDd3RnI0YWVuRkxhckZHVXN2bUErUlYwSHpJblk5YVZuaC9JWjJULzRDbEVtMlUwc3BXRU0zUE1OcFVTNWptdWFRVmU="
}

In [ ]:

# 以手動更新取得的urls，再利用 requests 取得於實價登錄網站取回 JSON 資料並回傳 DataFrame
def fetch_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # 若有錯誤狀況，會引發例外
        data = response.json()
        return pd.DataFrame(data)
    except Exception as e:
        print(f"取得資料時發生錯誤：{e}")
        return pd.DataFrame()  # 回傳空的 DataFrame

# 建立一個空的列表存放各區的 DataFrame
df_list = []

# 迴圈走訪所有 URL，並新增一個表示地區的欄位
for region, url in tqdm(urls.items(), desc="Fetching data"):
    df_temp = fetch_data(url)
    if not df_temp.empty:
        df_temp["region"] = region  # 加入來源區域欄位，便於後續分析
    df_list.append(df_temp)
    time.sleep(5)  # 每次發出請求後暫停 5 秒

# 利用 pd.concat 合併所有 DataFrame（重置索引）
combined_df = pd.concat(df_list, ignore_index=True)

# 顯示結果 (例如：顯示前 5 筆資料)
combined_df


Fetching data: 100%|██████████| 1/1 [00:06<00:00,  6.67s/it]


,AA11,addr,apply,applydate,b,chkdate,city,e,house,id,...,lon,ma,mark,name,pimg,pu,sn,subid,town,region
0,第三種住宅區,大安區臥龍街268號,潤泰創新國際股份有限公司,1100702,辛亥段四小段482地號,1100819,A,"自售:1100701~自完售止,1100701~售完為止;",130,G1A011007020007,...,118.754566,鋼筋混凝土造,潤泰創新國際股份有限公司,潤泰大安富陽,build_case.png,店舖、住宅,0,00,A02,臺北市
1,商業區,大同區延平北路一段25號旁,亞昕國際開發股份有限公司,1100702,"玉泉段一小段874地號,玉泉段一小段875地號,玉泉段一小段885地號,玉泉段一小段886地...",1100825,A,"自售:1090601~1091231,事務所;代銷:1100501~1111130",130,G2A011007020003,...,121.512361,鋼骨RC造,中國建築經理股份有限公司,亞昕華威,build_case.png,住商用,1,00,A09,臺北市
2,敦化南北路特定專用區(B區),松山區敦化北路56號,國美投資股份有限公司等5人,1100705,"敦化段二小段4地號,敦化段二小段4-1地號,敦化段二小段16地號,敦化段二小段32地號,敦化...",1120731,A,自售:無;代銷:110年10月21日起至使照取得6個月止,65,G2A011007050003,...,121.548457,鋼骨造,台新建築經理股份有限公司,敦仰,build_case.png,一般零售業甲組(店舖);集合住宅,2,02,A01,臺北市
3,第三種住宅區,士林區德行東路335號,朋記建設股份有限公司,1100705,"芝蘭段一小段320地號,芝蘭段一小段321地號,芝蘭段一小段325地號,芝蘭段一小段326地...",1101028,A,自售:無;代銷:1100701~1101231,27,G1A011007050001,...,121.537836,鋼筋混凝土造,台灣新光商業銀行股份有限公司,原禾賦,build_case.png,住商用,3,03,A15,臺北市
4,敦化南北路特定專用區(A)區,大安區建安里敦化南路一段161巷5弄6號2樓,富邦建設股份有限公司,1100705,"仁愛段六小段244地號,仁愛段六小段263-4地號,仁愛段六小段516地號,仁愛段六小段51...",1120721,A,自售:至完銷;代銷:1100301~1101031,60,G1A011007050002,...,121.549528,鋼骨造,富邦建設股份有限公司,富邦藝樹,build_case.png,住商用,4,02,A02,臺北市
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,第三種住宅區,松山區八德路三段12巷7弄23號、八德路三段12巷7弄23之1號,山輝建設股份有限公司,1140123,"延吉段三小段567地號,延吉段三小段568地號,延吉段三小段569地號,延吉段三小段570-...",1140326,A,自售:1140301~1160228;,66,G1A011401230001,...,121.551167,RC造,山輝建設股份有限公司 負責人：劉朝棟,三輝敦?,build_case.png,住商用,465,00,A01,臺北市
466,特定專用區(供一般商業使用),大同區永樂里延平北路二段36巷1號共34筆,世座建設股份有限公司,1140123,"迪化段二小段254地號,迪化段二小段255地號,迪化段二小段264地號,迪化段二小段265地...",1140225,A,自售:無;代銷:企劃銷售期間為自建照取得公開銷售後9個月，雙方得協議延長之。,238,G1A011401230002,...,121.511588,鋼骨RC造，RC造(供公眾使用建築物),世座建設股份有限公司 負責人：陳世昌,世座友樂町,build_case.png,住商用,466,00,A09,臺北市
467,第三種住宅區,南港區南港路一段175之1號1F 共26筆,築源建設股份有限公司,1140221,"南港段二小段528地號,南港段二小段529地號,南港段二小段530地號,南港段二小段531地...",,A,自售:1140301~1201214;,132,G1A011402210001,...,121.612517,鋼骨造,築源建設股份有限公司,築源南港-東御苑,build_case.png,住商用,467,00,A13,臺北市
468,第參種工業區(不得作住宅使用),內湖區週美里新明路174巷14號,耀達建設股份有限公司,1140224,潭美段三小段726地號,1140321,A,自售:1140224~1191231;,9,G1A011402240001,...,121.584952,RC造,耀達建設股份有限公司 負責人:莊文欽,金矽谷29期,build_case.png,辦公、服務類（G類）,468,00,A14,臺北市


In [15]:
combined_df.columns

Index(['AA11', 'addr', 'apply', 'applydate', 'b', 'chkdate', 'city', 'e',
       'house', 'id', 'idlist', 'lat', 'ldate', 'license', 'lon', 'ma', 'mark',
       'name', 'pimg', 'pu', 'sn', 'subid', 'town', 'region'],
      dtype='object')

In [27]:
new_column_names = {
    'AA11': '使用分區',
    'addr': '坐落街道',
    'apply': '申請人',
    'applydate': '申請備查期間',
    'b': '坐落基地',
    'chkdate': '備查完成日期',
    'city': '縣市代號',
    'e': '銷售期間',
    'house': '戶數',
    'id': '編號',
    'idlist': '編號列表',
    'lat': '緯度',
    'ldate': '建照核發日',
    'license': '建照執照',
    'lon': '經度',
    'ma': '主要建材',
    'mark': '起造人',
    'name': '社區名稱',
    'pimg': '圖片路徑',
    'pu': '主要用途',
    'sn': '序號',
    'subid': '子編號',
    'town': '行政區代號',
    'region': '縣市'
}

In [28]:
df = combined_df.rename(columns=new_column_names, inplace=False)
df

,使用分區,坐落街道,申請人,申請備查期間,坐落基地,備查完成日期,縣市代號,銷售期間,戶數,編號,...,經度,主要建材,起造人,社區名稱,圖片路徑,主要用途,序號,子編號,行政區代號,縣市
0,第三種住宅區,大安區臥龍街268號,潤泰創新國際股份有限公司,1100702,辛亥段四小段482地號,1100819,A,"自售:1100701~自完售止,1100701~售完為止;",130,G1A011007020007,...,118.754566,鋼筋混凝土造,潤泰創新國際股份有限公司,潤泰大安富陽,build_case.png,店舖、住宅,0,00,A02,臺北市
1,商業區,大同區延平北路一段25號旁,亞昕國際開發股份有限公司,1100702,"玉泉段一小段874地號,玉泉段一小段875地號,玉泉段一小段885地號,玉泉段一小段886地...",1100825,A,"自售:1090601~1091231,事務所;代銷:1100501~1111130",130,G2A011007020003,...,121.512361,鋼骨RC造,中國建築經理股份有限公司,亞昕華威,build_case.png,住商用,1,00,A09,臺北市
2,敦化南北路特定專用區(B區),松山區敦化北路56號,國美投資股份有限公司等5人,1100705,"敦化段二小段4地號,敦化段二小段4-1地號,敦化段二小段16地號,敦化段二小段32地號,敦化...",1120731,A,自售:無;代銷:110年10月21日起至使照取得6個月止,65,G2A011007050003,...,121.548457,鋼骨造,台新建築經理股份有限公司,敦仰,build_case.png,一般零售業甲組(店舖);集合住宅,2,02,A01,臺北市
3,第三種住宅區,士林區德行東路335號,朋記建設股份有限公司,1100705,"芝蘭段一小段320地號,芝蘭段一小段321地號,芝蘭段一小段325地號,芝蘭段一小段326地...",1101028,A,自售:無;代銷:1100701~1101231,27,G1A011007050001,...,121.537836,鋼筋混凝土造,台灣新光商業銀行股份有限公司,原禾賦,build_case.png,住商用,3,03,A15,臺北市
4,敦化南北路特定專用區(A)區,大安區建安里敦化南路一段161巷5弄6號2樓,富邦建設股份有限公司,1100705,"仁愛段六小段244地號,仁愛段六小段263-4地號,仁愛段六小段516地號,仁愛段六小段51...",1120721,A,自售:至完銷;代銷:1100301~1101031,60,G1A011007050002,...,121.549528,鋼骨造,富邦建設股份有限公司,富邦藝樹,build_case.png,住商用,4,02,A02,臺北市
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7973,第一種商業區,花蓮市富裕三街,上寬建設不動產有限公司,1140310,裕民段562地號,1140320,U,自售:備查完成至售完為止;代銷:備查完成至售完為止,23,G2UA11403100001,...,121.598333,鋼筋混凝土造,上寬建設不動產有限公司,上寬好室,build_case.png,集合住宅,146,00,U01,花蓮縣
7974,一般農業區丁種建築用地,新城鄉大德街,連科金興業有限公司,1140320,大漢段36地號,1140325,U,自售:無;代銷:1140501~1240501,26,G1UA11403200001,...,121.609311,鋼筋混凝土造,連科金興業有限公司,日安寬庭,build_case.png,工廠,147,00,U04,花蓮縣
7975,住宅區,花蓮市國富五街與國富八街口,星耀建設有限公司,1140324,莊敬段601地號,1140325,U,自售:自備查完成日起至銷售完成日訖;,16,G1UA11403240002,...,121.597978,鋼筋混凝土造,星耀建設有限公司,小倆口,build_case.png,住家用,148,00,U01,花蓮縣
7976,住宅區,吉安鄉北昌三街18巷11弄3號,德寶建設開發有限公司,1140325,"自強段283-1地號,自強段302地號,自強段309地號",1140407,U,自售:自備查完成後至銷售完畢;,20,G1UA11403250001,...,121.593886,鋼筋混凝土造,德寶建設開發有限公司,北昌學府,build_case.png,H2-集合住宅,149,00,U05,花蓮縣


In [29]:
selected_df = df[["縣市", "行政區代號", "社區名稱", "戶數", "銷售期間", "坐落街道", "主要用途", "使用分區", "備查完成日期", "建照核發日", "建照執照", "經度", "緯度"]]
selected_df

,縣市,行政區代號,社區名稱,戶數,銷售期間,坐落街道,主要用途,使用分區,備查完成日期,建照核發日,建照執照,經度,緯度
0,臺北市,A02,潤泰大安富陽,130,"自售:1100701~自完售止,1100701~售完為止;",大安區臥龍街268號,店舖、住宅,第三種住宅區,1100819,1080911,108建字第0168號,118.754566,0.000000
1,臺北市,A09,亞昕華威,130,"自售:1090601~1091231,事務所;代銷:1100501~1111130",大同區延平北路一段25號旁,住商用,商業區,1100825,1070103,107建字第0001號,121.512361,25.050456
2,臺北市,A01,敦仰,65,自售:無;代銷:110年10月21日起至使照取得6個月止,松山區敦化北路56號,一般零售業甲組(店舖);集合住宅,敦化南北路特定專用區(B區),1120731,1100611,110建字第0139號,121.548457,25.050931
3,臺北市,A15,原禾賦,27,自售:無;代銷:1100701~1101231,士林區德行東路335號,住商用,第三種住宅區,1101028,1080530,108建字第0100號,121.537836,25.110492
4,臺北市,A02,富邦藝樹,60,自售:至完銷;代銷:1100301~1101031,大安區建安里敦化南路一段161巷5弄6號2樓,住商用,敦化南北路特定專用區(A)區,1120721,1091029,109建字第0243號,121.549528,25.044184
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7973,花蓮縣,U01,上寬好室,23,自售:備查完成至售完為止;代銷:備查完成至售完為止,花蓮市富裕三街,集合住宅,第一種商業區,1140320,1140108,花建執照字第114A0006號,121.598333,23.993766
7974,花蓮縣,U04,日安寬庭,26,自售:無;代銷:1140501~1240501,新城鄉大德街,工廠,一般農業區丁種建築用地,1140325,1140109,花建執照字第114A0015號,121.609311,24.041740
7975,花蓮縣,U01,小倆口,16,自售:自備查完成日起至銷售完成日訖;,花蓮市國富五街與國富八街口,住家用,住宅區,1140325,1131231,花建執照字第113A0492號,121.597978,23.989352
7976,花蓮縣,U05,北昌學府,20,自售:自備查完成後至銷售完畢;,吉安鄉北昌三街18巷11弄3號,H2-集合住宅,住宅區,1140407,1130722,花建執照字第113A0283號,121.593886,23.986307


In [ ]:
# 由坐落地址欄位折分出行政區
def parse_admin_region(address):
    # 若不是字串或字串長度為0，直接回傳 None 或空值
    if not isinstance(address, str) or not address:
        return None
    
    # 判斷第二個字是否為「區」
    # 注意：Python 字串的索引從 0 開始
    if len(address) >= 2 and address[1] == "區":
        return address[:2]
    # 判斷第三個字是否為「區」
    elif len(address) >= 3 and address[2] == "區":
        return address[:3]
    # 其餘情況取前三個字
    elif len(address) >= 3:
        return address[:3]
    else:
        # 若字串不足三個字，就直接回傳原字串
        return address

In [ ]:
# 在 df 裡面新增一個「行政區」欄位
selected_df["行政區"] = selected_df["坐落街道"].apply(parse_admin_region)
selected_df.insert(selected_df.columns.get_loc("縣市") + 1, "行政區", selected_df.pop("行政區"))
selected_df

C:\Users\user\AppData\Local\Temp\ipykernel_19968\1743188186.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df["行政區"] = selected_df["坐落街道"].apply(parse_admin_region)


,縣市,行政區,行政區代號,社區名稱,戶數,銷售期間,坐落街道,主要用途,使用分區,備查完成日期,建照核發日,建照執照,經度,緯度
0,臺北市,大安區,A02,潤泰大安富陽,130,"自售:1100701~自完售止,1100701~售完為止;",大安區臥龍街268號,店舖、住宅,第三種住宅區,1100819,1080911,108建字第0168號,118.754566,0.000000
1,臺北市,大同區,A09,亞昕華威,130,"自售:1090601~1091231,事務所;代銷:1100501~1111130",大同區延平北路一段25號旁,住商用,商業區,1100825,1070103,107建字第0001號,121.512361,25.050456
2,臺北市,松山區,A01,敦仰,65,自售:無;代銷:110年10月21日起至使照取得6個月止,松山區敦化北路56號,一般零售業甲組(店舖);集合住宅,敦化南北路特定專用區(B區),1120731,1100611,110建字第0139號,121.548457,25.050931
3,臺北市,士林區,A15,原禾賦,27,自售:無;代銷:1100701~1101231,士林區德行東路335號,住商用,第三種住宅區,1101028,1080530,108建字第0100號,121.537836,25.110492
4,臺北市,大安區,A02,富邦藝樹,60,自售:至完銷;代銷:1100301~1101031,大安區建安里敦化南路一段161巷5弄6號2樓,住商用,敦化南北路特定專用區(A)區,1120721,1091029,109建字第0243號,121.549528,25.044184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7973,花蓮縣,花蓮市,U01,上寬好室,23,自售:備查完成至售完為止;代銷:備查完成至售完為止,花蓮市富裕三街,集合住宅,第一種商業區,1140320,1140108,花建執照字第114A0006號,121.598333,23.993766
7974,花蓮縣,新城鄉,U04,日安寬庭,26,自售:無;代銷:1140501~1240501,新城鄉大德街,工廠,一般農業區丁種建築用地,1140325,1140109,花建執照字第114A0015號,121.609311,24.041740
7975,花蓮縣,花蓮市,U01,小倆口,16,自售:自備查完成日起至銷售完成日訖;,花蓮市國富五街與國富八街口,住家用,住宅區,1140325,1131231,花建執照字第113A0492號,121.597978,23.989352
7976,花蓮縣,吉安鄉,U05,北昌學府,20,自售:自備查完成後至銷售完畢;,吉安鄉北昌三街18巷11弄3號,H2-集合住宅,住宅區,1140407,1130722,花建執照字第113A0283號,121.593886,23.986307


In [43]:
town_amt = selected_df['行政區'].value_counts()

for town, cnt in town_amt.items():
    print(f"{town}: {cnt}")

北屯區: 218
新竹市: 205
觀音區: 167
中壢區: 159
竹北市: 141
竹南鎮: 132
大園區: 128
桃園區: 122
嘉義市: 117
安南區: 105
楊梅區: 99
沙鹿區: 99
三重區: 93
西屯區: 92
頭份市: 89
台東市: 88
林口區: 86
永康區: 84
員林市: 81
湖口鄉: 79
南屯區: 78
善化區: 76
虎尾鎮: 76
五結鄉: 76
太平區: 74
龍井區: 73
北投區: 71
宜蘭市: 69
板橋區: 69
花蓮市: 68
中山區: 66
大安區: 65
斗六市: 65
梧棲區: 64
龜山區: 63
冬山鄉: 61
苗栗市: 61
蘆竹區: 60
土城區: 59
南區: 58
北區: 58
八德區: 57
新莊區: 56
淡水區: 55
東區: 52
南投市: 52
新豐鄉: 51
吉安鄉: 50
清水區: 49
新店區: 48
平鎮區: 47
新屋區: 46
後龍鎮: 46
新營區: 46
員山鄉: 45
佳里區: 45
士林區: 45
仁武區: 45
屏東市: 45
豐原區: 44
苑裡鎮: 44
大里區: 43
內湖區: 43
楠梓區: 43
彰化市: 43
龍潭區: 41
中和區: 41
后里區: 41
仁德區: 41
大同區: 41
二林鎮: 40
三民區: 39
新市區: 38
麻豆區: 38
壯圍鄉: 38
新埔鎮: 38
民雄鄉: 38
蘇澳鎮: 38
中正區: 38
歸仁區: 38
鳳山區: 37
安定區: 36
草屯鎮: 36
大寮區: 36
溪湖鎮: 36
礁溪鄉: 35
三星鄉: 33
烏日區: 33
芎林鄉: 33
神岡區: 33
潭子區: 31
和美鎮: 31
小港區: 31
左營區: 31
羅東鎮: 30
大甲區: 30
竹東鎮: 29
埔心鄉: 29
大雅區: 28
水上鄉: 28
林園區: 27
西螺鎮: 27
斗南鎮: 27
埔里鎮: 26
太保市: 26
文山區: 26
鶯歌區: 26
松山區: 26
北斗鎮: 26
埤頭鄉: 25
公館鄉: 25
五股區: 25
土庫鎮: 25
西區: 25
信義區: 24
岡山區: 24
萬華區: 24
前鎮區: 23
麥寮鄉: 23
安平區: 23
泰山區: 22
朴子市: 22
鼓山區: 22
關廟區: 22
南港

In [44]:
# 定義一個函式來解析銷售期間，回傳 (自售期間, 代銷期間)
def parse_sale_period(s: str):
    # 預設為 None 或空字串
    self_period = None
    agent_period = None

    # 利用正規表達式找出自售期間：匹配 "自售:" 後面所有字串，直到遇到 ";" 或 "代銷:" 或字串結尾
    self_match = re.search(r"自售:(.*?)(?=;|代銷:|$)", s)
    if self_match:
        self_period = self_match.group(1).strip()

    # 利用正規表達式找出代銷期間：匹配 "代銷:" 後面所有字串，直到遇到 ";" 或字串結尾
    agent_match = re.search(r"代銷:(.*?)(?=;|$)", s)
    if agent_match:
        agent_period = agent_match.group(1).strip()

    return self_period, agent_period

In [48]:
selected_df["自售期間"], selected_df["代銷期間"] = zip(*selected_df["銷售期間"].apply(parse_sale_period))
selected_df

C:\Users\user\AppData\Local\Temp\ipykernel_19968\3630583983.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df["自售期間"], selected_df["代銷期間"] = zip(*selected_df["銷售期間"].apply(parse_sale_period))


,縣市,行政區,行政區代號,社區名稱,戶數,銷售期間,坐落街道,主要用途,使用分區,備查完成日期,建照核發日,建照執照,經度,緯度,自售期間,代銷期間
0,臺北市,大安區,A02,潤泰大安富陽,130,"自售:1100701~自完售止,1100701~售完為止;",大安區臥龍街268號,店舖、住宅,第三種住宅區,1100819,1080911,108建字第0168號,118.754566,0.000000,"1100701~自完售止,1100701~售完為止",None
1,臺北市,大同區,A09,亞昕華威,130,"自售:1090601~1091231,事務所;代銷:1100501~1111130",大同區延平北路一段25號旁,住商用,商業區,1100825,1070103,107建字第0001號,121.512361,25.050456,"1090601~1091231,事務所",1100501~1111130
2,臺北市,松山區,A01,敦仰,65,自售:無;代銷:110年10月21日起至使照取得6個月止,松山區敦化北路56號,一般零售業甲組(店舖);集合住宅,敦化南北路特定專用區(B區),1120731,1100611,110建字第0139號,121.548457,25.050931,無,110年10月21日起至使照取得6個月止
3,臺北市,士林區,A15,原禾賦,27,自售:無;代銷:1100701~1101231,士林區德行東路335號,住商用,第三種住宅區,1101028,1080530,108建字第0100號,121.537836,25.110492,無,1100701~1101231
4,臺北市,大安區,A02,富邦藝樹,60,自售:至完銷;代銷:1100301~1101031,大安區建安里敦化南路一段161巷5弄6號2樓,住商用,敦化南北路特定專用區(A)區,1120721,1091029,109建字第0243號,121.549528,25.044184,至完銷,1100301~1101031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7973,花蓮縣,花蓮市,U01,上寬好室,23,自售:備查完成至售完為止;代銷:備查完成至售完為止,花蓮市富裕三街,集合住宅,第一種商業區,1140320,1140108,花建執照字第114A0006號,121.598333,23.993766,備查完成至售完為止,備查完成至售完為止
7974,花蓮縣,新城鄉,U04,日安寬庭,26,自售:無;代銷:1140501~1240501,新城鄉大德街,工廠,一般農業區丁種建築用地,1140325,1140109,花建執照字第114A0015號,121.609311,24.041740,無,1140501~1240501
7975,花蓮縣,花蓮市,U01,小倆口,16,自售:自備查完成日起至銷售完成日訖;,花蓮市國富五街與國富八街口,住家用,住宅區,1140325,1131231,花建執照字第113A0492號,121.597978,23.989352,自備查完成日起至銷售完成日訖,None
7976,花蓮縣,吉安鄉,U05,北昌學府,20,自售:自備查完成後至銷售完畢;,吉安鄉北昌三街18巷11弄3號,H2-集合住宅,住宅區,1140407,1130722,花建執照字第113A0283號,121.593886,23.986307,自備查完成後至銷售完畢,None


In [49]:
# 1. 先記錄「銷售期間」欄位的位置
sales_period_idx = selected_df.columns.get_loc("銷售期間")

# 2. 將「自售期間」和「代銷期間」欄位取出 (pop 出來的同時也從 DataFrame 移除)
self_period = selected_df.pop("自售期間")
agent_period = selected_df.pop("代銷期間")

# 3. 利用 insert 方法依序將它們插入到「銷售期間」欄位後面
selected_df.insert(sales_period_idx + 1, "自售期間", self_period)
selected_df.insert(sales_period_idx + 2, "代銷期間", agent_period)

selected_df

,縣市,行政區,行政區代號,社區名稱,戶數,銷售期間,自售期間,代銷期間,坐落街道,主要用途,使用分區,備查完成日期,建照核發日,建照執照,經度,緯度
0,臺北市,大安區,A02,潤泰大安富陽,130,"自售:1100701~自完售止,1100701~售完為止;","1100701~自完售止,1100701~售完為止",None,大安區臥龍街268號,店舖、住宅,第三種住宅區,1100819,1080911,108建字第0168號,118.754566,0.000000
1,臺北市,大同區,A09,亞昕華威,130,"自售:1090601~1091231,事務所;代銷:1100501~1111130","1090601~1091231,事務所",1100501~1111130,大同區延平北路一段25號旁,住商用,商業區,1100825,1070103,107建字第0001號,121.512361,25.050456
2,臺北市,松山區,A01,敦仰,65,自售:無;代銷:110年10月21日起至使照取得6個月止,無,110年10月21日起至使照取得6個月止,松山區敦化北路56號,一般零售業甲組(店舖);集合住宅,敦化南北路特定專用區(B區),1120731,1100611,110建字第0139號,121.548457,25.050931
3,臺北市,士林區,A15,原禾賦,27,自售:無;代銷:1100701~1101231,無,1100701~1101231,士林區德行東路335號,住商用,第三種住宅區,1101028,1080530,108建字第0100號,121.537836,25.110492
4,臺北市,大安區,A02,富邦藝樹,60,自售:至完銷;代銷:1100301~1101031,至完銷,1100301~1101031,大安區建安里敦化南路一段161巷5弄6號2樓,住商用,敦化南北路特定專用區(A)區,1120721,1091029,109建字第0243號,121.549528,25.044184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7973,花蓮縣,花蓮市,U01,上寬好室,23,自售:備查完成至售完為止;代銷:備查完成至售完為止,備查完成至售完為止,備查完成至售完為止,花蓮市富裕三街,集合住宅,第一種商業區,1140320,1140108,花建執照字第114A0006號,121.598333,23.993766
7974,花蓮縣,新城鄉,U04,日安寬庭,26,自售:無;代銷:1140501~1240501,無,1140501~1240501,新城鄉大德街,工廠,一般農業區丁種建築用地,1140325,1140109,花建執照字第114A0015號,121.609311,24.041740
7975,花蓮縣,花蓮市,U01,小倆口,16,自售:自備查完成日起至銷售完成日訖;,自備查完成日起至銷售完成日訖,None,花蓮市國富五街與國富八街口,住家用,住宅區,1140325,1131231,花建執照字第113A0492號,121.597978,23.989352
7976,花蓮縣,吉安鄉,U05,北昌學府,20,自售:自備查完成後至銷售完畢;,自備查完成後至銷售完畢,None,吉安鄉北昌三街18巷11弄3號,H2-集合住宅,住宅區,1140407,1130722,花建執照字第113A0283號,121.593886,23.986307


In [50]:
# 定義函式：尋找第一個 7 位數字，若沒有則回傳 None
def find_first_seven_digits(text):
    if not isinstance(text, str):
        return None
    match = re.search(r"\d{7}", text)
    if match:
        return match.group(0)  # 取出第一個符合 7 位數字的字串
    return None

In [54]:
# 依規則從「自售期間」及「代銷期間」欄位提取出7位數字，
# 分別存入新欄位「自售起始時間」與「代銷起始時間」
selected_df["自售起始時間"] = selected_df["自售期間"].apply(find_first_seven_digits)
selected_df["代銷起始時間"] = selected_df["代銷期間"].apply(find_first_seven_digits)

# 將「自售起始時間」欄位移動到「自售期間」欄位後面：
self_time = selected_df.pop("自售起始時間")
self_period_pos = selected_df.columns.get_loc("自售期間")
selected_df.insert(self_period_pos + 1, "自售起始時間", self_time)

# 將「代銷起始時間」欄位移動到「代銷期間」欄位後面：
agent_time = selected_df.pop("代銷起始時間")
agent_period_pos = selected_df.columns.get_loc("代銷期間")
selected_df.insert(agent_period_pos + 1, "代銷起始時間", agent_time)

selected_df


C:\Users\user\AppData\Local\Temp\ipykernel_19968\2016884795.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df["自售起始時間"] = selected_df["自售期間"].apply(find_first_seven_digits)
C:\Users\user\AppData\Local\Temp\ipykernel_19968\2016884795.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df["代銷起始時間"] = selected_df["代銷期間"].apply(find_first_seven_digits)


,縣市,行政區,行政區代號,社區名稱,戶數,銷售期間,自售期間,自售起始時間,代銷期間,代銷起始時間,坐落街道,主要用途,使用分區,備查完成日期,建照核發日,建照執照,經度,緯度
0,臺北市,大安區,A02,潤泰大安富陽,130,"自售:1100701~自完售止,1100701~售完為止;","1100701~自完售止,1100701~售完為止",1100701,None,None,大安區臥龍街268號,店舖、住宅,第三種住宅區,1100819,1080911,108建字第0168號,118.754566,0.000000
1,臺北市,大同區,A09,亞昕華威,130,"自售:1090601~1091231,事務所;代銷:1100501~1111130","1090601~1091231,事務所",1090601,1100501~1111130,1100501,大同區延平北路一段25號旁,住商用,商業區,1100825,1070103,107建字第0001號,121.512361,25.050456
2,臺北市,松山區,A01,敦仰,65,自售:無;代銷:110年10月21日起至使照取得6個月止,無,None,110年10月21日起至使照取得6個月止,None,松山區敦化北路56號,一般零售業甲組(店舖);集合住宅,敦化南北路特定專用區(B區),1120731,1100611,110建字第0139號,121.548457,25.050931
3,臺北市,士林區,A15,原禾賦,27,自售:無;代銷:1100701~1101231,無,None,1100701~1101231,1100701,士林區德行東路335號,住商用,第三種住宅區,1101028,1080530,108建字第0100號,121.537836,25.110492
4,臺北市,大安區,A02,富邦藝樹,60,自售:至完銷;代銷:1100301~1101031,至完銷,None,1100301~1101031,1100301,大安區建安里敦化南路一段161巷5弄6號2樓,住商用,敦化南北路特定專用區(A)區,1120721,1091029,109建字第0243號,121.549528,25.044184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7973,花蓮縣,花蓮市,U01,上寬好室,23,自售:備查完成至售完為止;代銷:備查完成至售完為止,備查完成至售完為止,None,備查完成至售完為止,None,花蓮市富裕三街,集合住宅,第一種商業區,1140320,1140108,花建執照字第114A0006號,121.598333,23.993766
7974,花蓮縣,新城鄉,U04,日安寬庭,26,自售:無;代銷:1140501~1240501,無,None,1140501~1240501,1140501,新城鄉大德街,工廠,一般農業區丁種建築用地,1140325,1140109,花建執照字第114A0015號,121.609311,24.041740
7975,花蓮縣,花蓮市,U01,小倆口,16,自售:自備查完成日起至銷售完成日訖;,自備查完成日起至銷售完成日訖,None,None,None,花蓮市國富五街與國富八街口,住家用,住宅區,1140325,1131231,花建執照字第113A0492號,121.597978,23.989352
7976,花蓮縣,吉安鄉,U05,北昌學府,20,自售:自備查完成後至銷售完畢;,自備查完成後至銷售完畢,None,None,None,吉安鄉北昌三街18巷11弄3號,H2-集合住宅,住宅區,1140407,1130722,花建執照字第113A0283號,121.593886,23.986307


In [56]:
# 假設 selected_df 已存在，且包含必需欄位：
# 「自售起始時間」、「代銷起始時間」、「銷售期間」、「備查完成日期」、「建照核發日」與「戶數」

# 定義函式：依據規則決定「銷售起始時間」
def compute_sales_start_time(row):
    self_time = row["自售起始時間"]
    agent_time = row["代銷起始時間"]
    
    # Rule 1: 若其中一個有值、另一個為空，則取有值的那一個
    if pd.isna(self_time) and not pd.isna(agent_time):
        return agent_time
    elif pd.isna(agent_time) and not pd.isna(self_time):
        return self_time
    # Rule 2: 如果兩者皆有值，轉為數值比較，取較小者（轉回字串）
    elif not pd.isna(self_time) and not pd.isna(agent_time):
        try:
            self_val = int(self_time)
            agent_val = int(agent_time)
            return str(min(self_val, agent_val))
        except Exception:
            return ""
    # Rule 3: 如果兩者皆無值，則檢查「銷售期間」是否包含"備查"
    elif pd.isna(self_time) and pd.isna(agent_time):
        sales_period = row["銷售期間"]
        if isinstance(sales_period, str) and "備查" in sales_period:
            return row["備查完成日期"]
        else:
            return row["建照核發日"]
    # Rule 4: 其餘情況，回傳空值
    else:
        return ""

# 建立「銷售起始時間」欄位
selected_df["銷售起始時間"] = selected_df.apply(compute_sales_start_time, axis=1)

# 將「銷售起始時間」欄位移至「戶數」欄位之後
sales_start = selected_df.pop("銷售起始時間")
w_index = selected_df.columns.get_loc("戶數")
selected_df.insert(w_index + 1, "銷售起始時間", sales_start)

selected_df

C:\Users\user\AppData\Local\Temp\ipykernel_19968\2409938647.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df["銷售起始時間"] = selected_df.apply(compute_sales_start_time, axis=1)


,縣市,行政區,行政區代號,社區名稱,戶數,銷售起始時間,銷售期間,自售期間,自售起始時間,代銷期間,代銷起始時間,坐落街道,主要用途,使用分區,備查完成日期,建照核發日,建照執照,經度,緯度
0,臺北市,大安區,A02,潤泰大安富陽,130,1100701,"自售:1100701~自完售止,1100701~售完為止;","1100701~自完售止,1100701~售完為止",1100701,None,None,大安區臥龍街268號,店舖、住宅,第三種住宅區,1100819,1080911,108建字第0168號,118.754566,0.000000
1,臺北市,大同區,A09,亞昕華威,130,1090601,"自售:1090601~1091231,事務所;代銷:1100501~1111130","1090601~1091231,事務所",1090601,1100501~1111130,1100501,大同區延平北路一段25號旁,住商用,商業區,1100825,1070103,107建字第0001號,121.512361,25.050456
2,臺北市,松山區,A01,敦仰,65,1100611,自售:無;代銷:110年10月21日起至使照取得6個月止,無,None,110年10月21日起至使照取得6個月止,None,松山區敦化北路56號,一般零售業甲組(店舖);集合住宅,敦化南北路特定專用區(B區),1120731,1100611,110建字第0139號,121.548457,25.050931
3,臺北市,士林區,A15,原禾賦,27,1100701,自售:無;代銷:1100701~1101231,無,None,1100701~1101231,1100701,士林區德行東路335號,住商用,第三種住宅區,1101028,1080530,108建字第0100號,121.537836,25.110492
4,臺北市,大安區,A02,富邦藝樹,60,1100301,自售:至完銷;代銷:1100301~1101031,至完銷,None,1100301~1101031,1100301,大安區建安里敦化南路一段161巷5弄6號2樓,住商用,敦化南北路特定專用區(A)區,1120721,1091029,109建字第0243號,121.549528,25.044184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7973,花蓮縣,花蓮市,U01,上寬好室,23,1140320,自售:備查完成至售完為止;代銷:備查完成至售完為止,備查完成至售完為止,None,備查完成至售完為止,None,花蓮市富裕三街,集合住宅,第一種商業區,1140320,1140108,花建執照字第114A0006號,121.598333,23.993766
7974,花蓮縣,新城鄉,U04,日安寬庭,26,1140501,自售:無;代銷:1140501~1240501,無,None,1140501~1240501,1140501,新城鄉大德街,工廠,一般農業區丁種建築用地,1140325,1140109,花建執照字第114A0015號,121.609311,24.041740
7975,花蓮縣,花蓮市,U01,小倆口,16,1140325,自售:自備查完成日起至銷售完成日訖;,自備查完成日起至銷售完成日訖,None,None,None,花蓮市國富五街與國富八街口,住家用,住宅區,1140325,1131231,花建執照字第113A0492號,121.597978,23.989352
7976,花蓮縣,吉安鄉,U05,北昌學府,20,1140407,自售:自備查完成後至銷售完畢;,自備查完成後至銷售完畢,None,None,None,吉安鄉北昌三街18巷11弄3號,H2-集合住宅,住宅區,1140407,1130722,花建執照字第113A0283號,121.593886,23.986307


In [60]:
final_df = selected_df[["縣市", "行政區", "社區名稱", "戶數", "銷售起始時間", "銷售期間",  "自售起始時間", "代銷起始時間", "備查完成日期", "建照核發日", "坐落街道", "主要用途", "使用分區", "建照執照", "經度", "緯度"]]
final_df

,縣市,行政區,社區名稱,戶數,銷售起始時間,銷售期間,自售起始時間,代銷起始時間,備查完成日期,建照核發日,坐落街道,主要用途,使用分區,建照執照,經度,緯度
0,臺北市,大安區,潤泰大安富陽,130,1100701,"自售:1100701~自完售止,1100701~售完為止;",1100701,None,1100819,1080911,大安區臥龍街268號,店舖、住宅,第三種住宅區,108建字第0168號,118.754566,0.000000
1,臺北市,大同區,亞昕華威,130,1090601,"自售:1090601~1091231,事務所;代銷:1100501~1111130",1090601,1100501,1100825,1070103,大同區延平北路一段25號旁,住商用,商業區,107建字第0001號,121.512361,25.050456
2,臺北市,松山區,敦仰,65,1100611,自售:無;代銷:110年10月21日起至使照取得6個月止,None,None,1120731,1100611,松山區敦化北路56號,一般零售業甲組(店舖);集合住宅,敦化南北路特定專用區(B區),110建字第0139號,121.548457,25.050931
3,臺北市,士林區,原禾賦,27,1100701,自售:無;代銷:1100701~1101231,None,1100701,1101028,1080530,士林區德行東路335號,住商用,第三種住宅區,108建字第0100號,121.537836,25.110492
4,臺北市,大安區,富邦藝樹,60,1100301,自售:至完銷;代銷:1100301~1101031,None,1100301,1120721,1091029,大安區建安里敦化南路一段161巷5弄6號2樓,住商用,敦化南北路特定專用區(A)區,109建字第0243號,121.549528,25.044184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7973,花蓮縣,花蓮市,上寬好室,23,1140320,自售:備查完成至售完為止;代銷:備查完成至售完為止,None,None,1140320,1140108,花蓮市富裕三街,集合住宅,第一種商業區,花建執照字第114A0006號,121.598333,23.993766
7974,花蓮縣,新城鄉,日安寬庭,26,1140501,自售:無;代銷:1140501~1240501,None,1140501,1140325,1140109,新城鄉大德街,工廠,一般農業區丁種建築用地,花建執照字第114A0015號,121.609311,24.041740
7975,花蓮縣,花蓮市,小倆口,16,1140325,自售:自備查完成日起至銷售完成日訖;,None,None,1140325,1131231,花蓮市國富五街與國富八街口,住家用,住宅區,花建執照字第113A0492號,121.597978,23.989352
7976,花蓮縣,吉安鄉,北昌學府,20,1140407,自售:自備查完成後至銷售完畢;,None,None,1140407,1130722,吉安鄉北昌三街18巷11弄3號,H2-集合住宅,住宅區,花建執照字第113A0283號,121.593886,23.986307
